In [2]:
import pandas as pd
import numpy as np
data = pd.read_excel('weather.xls', usecols=range(1,6))

In [3]:
data

,outlook,temp.,hum.,windy,class
0,sunny,85,85,False,Don’t Play
1,sunny,80,90,True,Don’t Play
2,overcast,83,78,False,Play
3,rain,70,96,False,Play
4,rain,68,80,False,Play
5,rain,65,70,True,Don’t Play
6,overcast,64,65,True,Play
7,sunny,72,95,False,Don’t Play
8,sunny,69,70,False,Play
9,rain,75,80,False,Play


In [4]:
data = data.sort_values(by=['temp.'])

In [5]:
data

,outlook,temp.,hum.,windy,class
6,overcast,64,65,True,Play
5,rain,65,70,True,Don’t Play
4,rain,68,80,False,Play
8,sunny,69,70,False,Play
3,rain,70,96,False,Play
13,rain,71,80,True,Don’t Play
7,sunny,72,95,False,Don’t Play
11,outlook,72,90,True,Play
9,rain,75,80,False,Play
10,sunny,75,70,True,Play


In [6]:
import math
from copy import deepcopy
from collections import Counter

In [7]:
def transform_set_into_list(class_set):
    class_list = []
    for key, value in class_set.items():
        class_list.append( (key, value))
    return class_list


In [24]:
num_attr_name = 'temp.'
class_attr_name = 'class'
discr_result = []
df_len = len(data)
def find_most_freq_class(bucket):
    #structure of bin/basket: [(index, num_val, class)]
    set_class = []
    for val in bucket:
        set_class.append(val[2])
    from collections import Counter
    unique_val = list(Counter(set_class).keys())
    count_val = list(Counter(set_class).values())
    ind_max = np.argmax(count_val)
    #if there`s several equal max 
    for idx, val in enumerate(count_val):
        #if there`s val that equals to our defined and have another index
        if val == count_val[ind_max] and idx != ind_max:
            print(count_val[ind_max])
            return False
    return unique_val[ind_max]

def eval_entropy(class_set):
    values_count = []
    given_split_count = class_set.count()
    entropy = 0
    for val in class_set.value_counts():
        values_count.append(val)
        entropy += -val/given_split_count * math.log2(val/given_split_count) 
    entropy *= given_split_count / df_len
    return entropy

def all_the_same(elements):
    if len(set(elements)) == 1: 
        return True
    
def entropy_based_discr(data):
    #CHECK WHETHER THE SAME NUMERIC VALUES
    #BEGIN
    numeric_vals = []
    num_attr_idx = data.columns.get_loc(num_attr_name)
    for index, row in data.iterrows():
        numeric_vals.append(row[num_attr_idx])
    if all_the_same(numeric_vals):
        interval = []
        num_attr_idx = data.columns.get_loc(num_attr_name)
        class_attr_idx = data.columns.get_loc(class_attr_name)
        for index, row in data.iterrows():
            interval.append( (row[num_attr_idx], row[class_attr_idx]))
        discr_result.append(interval)
        return
    #END
    
    
    
    if eval_entropy(data[class_attr_name]) == 0:
        interval = []
        num_attr_idx = data.columns.get_loc(num_attr_name)
        class_attr_idx = data.columns.get_loc(class_attr_name)
        for index, row in data.iterrows():
            interval.append( (row[num_attr_idx], row[class_attr_idx]))
        discr_result.append(interval)
        return
    min_entropy = 'unsetted'
    #print(data[num_attr_name], "asd")
    for val in data[num_attr_name]:
        split_point = val+0.5
        first_part = data[data[num_attr_name] < split_point][class_attr_name]
        second_part = data[data[num_attr_name] > split_point][class_attr_name]
        first_part_entropy = eval_entropy(first_part)
        second_part_entropy = eval_entropy(second_part)
        entropy =  first_part_entropy + second_part_entropy
        if min_entropy == 'unsetted':
            min_entropy = (split_point, entropy)
        elif min_entropy[1] > entropy:
            min_entropy = (split_point, entropy)
    first_part = data[data[num_attr_name] < min_entropy[0]]
    second_part = data[data[num_attr_name] > min_entropy[0]]
    #print(first_part, 'XYD')
    #print(second_part, 'XYZ')
    #print("\n")
    entropy_based_discr(first_part)
    entropy_based_discr(second_part)
entropy_based_discr(data)

In [23]:
discr_result

[[(64, 'Play')],
 [(65, 'Don’t Play')],
 [(68, 'Play'), (69, 'Play'), (70, 'Play')],
 [(71, 'Don’t Play')],
 [(72, 'Don’t Play'), (72, 'Play')],
 [(75, 'Play'), (75, 'Play')],
 [(80, 'Don’t Play')],
 [(81, 'Play'), (83, 'Play')],
 [(85, 'Don’t Play')]]

In [20]:
data.iloc[1]

outlook          rain
temp.              65
hum.               70
windy            True
class      Don’t Play
Name: 5, dtype: object

In [11]:
sd = data[data['temp.'] > 83]

In [12]:
sd

,outlook,temp.,hum.,windy,class
0,sunny,85,85,False,Don’t Play


In [13]:
for index, row in sd.iterrows():
    print(row)

outlook         sunny
temp.              85
hum.               85
windy           False
class      Don’t Play
Name: 0, dtype: object


In [14]:
eval_entropy(data[data['temp.'] > 83]['class'])

0    Don’t Play
Name: class, dtype: object


0.0